In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.events import EventFiringWebDriver, AbstractEventListener
from selenium.webdriver import Firefox
import time
from selenium.webdriver.common.action_chains import ActionChains

df=pd.DataFrame(columns=['route_name','route_link','bus_name','bus_type','departing_time','duration','reaching_time','star_rating','price','seats_available'])
df

,route_name,route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seats_available


In [6]:
travels_links_dict = {}

driver = webdriver.Firefox()
driver.get('https://www.redbus.in/')
driver.maximize_window()

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'rtcName')))
popular_bus_depts = driver.find_elements(By.CLASS_NAME, 'rtcName')
view_buses_buttons = driver.find_element(By.XPATH, "/html/body/section/div[2]/main/div[3]/div[3]/div[1]/div[2]/a")
popular_bus_names = [bus.text for bus in popular_bus_depts] 
driver.get(view_buses_buttons.get_attribute('href'))
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'D113_link')))
bus_depts = driver.find_elements(By.CLASS_NAME, 'D113_link')
d113_bus_dict={}
for bus in bus_depts:
    for name in popular_bus_names:
        if name in bus.text:
            d113_bus_dict[name]=bus.get_attribute('href')
driver.quit()

In [7]:
driver=webdriver.Firefox()
driver.maximize_window()
for name_string, link in d113_bus_dict.items():
    try:
        driver.get(link)
        time.sleep(2)
        
        # Wait for the route elements to be present
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "route"))
        )
        
        try:
            # Check if pagination exists
            parent_div = driver.find_element(By.CLASS_NAME, "DC_117_paginationTable")
            
            try:
                # Try to find pagination buttons
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.DC_117_pageTabs"))
                )
                
                child_count = driver.execute_script(
                    "return arguments[0].getElementsByTagName('div').length;", parent_div
                )
                
                # If buttons are found, iterate through pagination
                for i in range(1, child_count + 1):
                    button = driver.find_element(By.CSS_SELECTOR, f"div.DC_117_pageTabs:nth-child({i})")
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
                    driver.execute_script("arguments[0].click();", button)
                    
                    # Extract route details on each page
                    route_details = WebDriverWait(driver, 5).until(
                        EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
                    )
                    for route in route_details:
                        route_link = route.get_attribute('href')
                        travels_links_dict[name_string + '_' + route.text] = route_link
                    
                    time.sleep(1)
                    
            except TimeoutException:
                # If buttons aren't found but parent div exists, get routes from current page
                print(f"Pagination buttons not found for link: {link}. Extracting from current page...")
                route_details = driver.find_elements(By.CLASS_NAME, 'route')
                for route in route_details:
                    route_link = route.get_attribute('href')
                    travels_links_dict[name_string + '_' + route.text] = route_link
                    
        except NoSuchElementException:
            # If pagination table doesn't exist, get routes from current page
            print(f"No pagination found for link: {link}. Extracting from current page...")
            route_details = driver.find_elements(By.CLASS_NAME, 'route')
            for route in route_details:
                route_link = route.get_attribute('href')
                travels_links_dict[name_string + '_' + route.text] = route_link
                
    except Exception as e:
        print(f"Error occurred while processing link: {link}. Skipping {name_string}")
        continue  # Skip to the next link
driver.quit()

Error occurred while processing link: https://www.redbus.in/online-booking/bsrtc-operated-by-vip-travels. Skipping... Exception: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Error occurred while processing link: https://www.redbus.in/online-booking/pepsu-punjab. Skipping... Exception: name 'NoSuchElementException' is not defined


In [8]:
travels_links_dict

{'APSRTC_Hyderabad to Vijayawada': 'https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada',
 'APSRTC_Vijayawada to Hyderabad': 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad',
 'APSRTC_Hyderabad to Ongole': 'https://www.redbus.in/bus-tickets/hyderabad-to-ongole',
 'APSRTC_Ongole to Hyderabad': 'https://www.redbus.in/bus-tickets/ongole-to-hyderabad',
 'APSRTC_Kakinada to Visakhapatnam': 'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam',
 'APSRTC_Bangalore to Tirupati': 'https://www.redbus.in/bus-tickets/bangalore-to-tirupathi',
 'APSRTC_Bangalore to Kadapa': 'https://www.redbus.in/bus-tickets/bangalore-to-kadapa',
 'APSRTC_Hyderabad to Narasaraopet': 'https://www.redbus.in/bus-tickets/hyderabad-to-narsaraopet',
 'APSRTC_Chittoor (Andhra Pradesh) to Bangalore': 'https://www.redbus.in/bus-tickets/chittoor-andhra-pradesh-to-bangalore',
 'APSRTC_Visakhapatnam to Kakinada': 'https://www.redbus.in/bus-tickets/visakhapatnam-to-kakinada',
 'APSRTC_Narasaraopet to Hyd

In [111]:
driver=webdriver.Firefox()
driver.maximize_window()
for route_name, route_link in travels_links_dict.items():
    driver.get(route_link)
    driver.maximize_window()
    time.sleep(5)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'button')))
    driver.back()
    body=driver.find_element(By.TAG_NAME,"body")
    body.send_keys(Keys.PAGE_DOWN)
    view_buses_buttons = [driver.find_elements(By.CLASS_NAME, "button")]
    for button in view_buses_buttons[0]:
        if button.text == "View Buses" or button.text == "VIEW BUSES":
            print(button.text)
            button.click()
            print(button.text)  
    scrolling=True
    while scrolling:
        old_page_source=driver.page_source
        body.send_keys(Keys.ARROW_UP)
        new_page_source=driver.page_source
        if new_page_source==old_page_source:
            scrolling=False
    scrolling=True
    while scrolling:
        old_page_source=driver.page_source
        body.send_keys(Keys.CONTROL + Keys.END)
        time.sleep(1)
        new_page_source=driver.page_source
        if new_page_source==old_page_source:
            scrolling=False
                        
    scrolling=True
    while scrolling:
        old_page_source=driver.page_source
        body.send_keys(Keys.ARROW_UP)
        new_page_source=driver.page_source
        if new_page_source==old_page_source:
            scrolling=False
        bus_names=driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
        bus_types=driver.find_elements(By.XPATH, ".//div[contains(@class, 'bus-type')]")
        departing_time=driver.find_elements(By.XPATH, ".//div[contains(@class, 'dp-time')]")
        duration = driver.find_elements(By.XPATH, ".//div[contains(@class, 'dur')]")    
        arrival_time = driver.find_elements(By.XPATH, ".//div[contains(@class, 'bp-time')]")
    break
        
    #     details_box = driver.find_elements(By.XPATH, '/html/body/section/div[2]/div[4]/div/div[2]/div/div[2]/div[2]/div[3]/ul/div/li/div[1]/div/div[1]/div[1]')
    #     for element in details_box:
    #         bus_name = element.find_element(By.XPATH, ".//div[contains(@class, 'travels')]").text
    #         bus_type = element.find_element(By.XPATH, ".//div[contains(@class, 'bus-type')]").text
    #         departing_time = element.find_element(By.XPATH, ".//div[contains(@class, 'dp-time')]").text
    #         duration = element.find_element(By.XPATH, ".//div[contains(@class, 'dur')]").text
    
    # # Combining arrival time and date
    #         arrival_time = element.find_element(By.XPATH, ".//div[contains(@class, 'bp-time')]").text
    #         # arrival_date = element.find_element(By.XPATH, ".//div[contains(@class, 'next-day-dp-lbl')]").text
    #         # reaching_time = f"{arrival_time}"
    
    # # Print or process the extracted information
    #         print(f"Bus Name: {bus_name}")
    #         # print(f"Bus Type: {bus_type}")
    #         # print(f"Departure Time: {departing_time}")
    #         # print(f"Duration: {duration}")
    #         # print(f"Reaching Time: {arrival_time}")
    #         # print("-" * 50)
    df=pd.DataFrame(columns=['route_name','route_link','bus_name','bus_type','departing_time','duration','reaching_time','star_rating','price','seats_available'])

        

VIEW BUSES
HIDE BUSES
VIEW BUSES
HIDE BUSES


In [114]:
for index,bus in enumerate(bus_names):
    print(f"{bus.text}")
    print(bus_types[index].text)
    print(departing_time[index].text)
    print(duration[index].text)
    print(arrival_time[index].text)

APSRTC - 2680
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
19:00
07h 00m
02:00
APSRTC - 2798
INDRA(A.C. Seater)
19:15
07h 15m
02:30
APSRTC - 2563
INDRA(A.C. Seater)
19:30
05h 55m
01:25
APSRTC - 2573
AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)
19:45
06h 45m
02:30
APSRTC - 2678
NIGHT RIDER (SEATER CUM SLEEPER)
20:00
07h 00m
03:00
APSRTC - 4880
STAR LINER(NON-AC SLEEPER 2+1)
20:00
06h 45m
02:45
APSRTC - 3622
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
20:00
08h 00m
04:00
APSRTC - 3977
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
20:15
08h 00m
04:15
APSRTC - 37801
INDRA(A.C. Seater)
20:20
07h 25m
03:45
APSRTC - 4179
AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)
20:30
07h 30m
04:00
APSRTC - 3962
INDRA(A.C. Seater)
20:40
07h 35m
04:15
APSRTC - 35212
STAR LINER(NON-AC SLEEPER 2+1)
20:45
07h 00m
03:45
APSRTC - 3544
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
20:45
07h 30m
04:15
APSRTC - 4795
INDRA(A.C. Seater)
20:50
07h 10m
04:00
APSRTC - 3883
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
20:50
07h 40m
04:30
APSRTC - 35064
SU